In [256]:
import pandas as pd
from tensorflow.keras.layers import LSTM, Flatten, Dense
from tensorflow.keras.models import Sequential
import keras
import tensorflow.keras.backend as K
import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn import preprocessing

In [366]:
VIX = pd.read_excel ('data_VIX.xlsx')
AGG = pd.read_excel ('data_AGG.xlsx')
DBC = pd.read_excel ('data_DBC.xlsx')
VTI = pd.read_excel ('data_VTI.xlsx')
all_data = pd.concat([VIX['Close'],AGG['Close'],DBC['Close'],VTI['Close']],axis=1)
all_data.columns = ['VIX','AGG','DBC','VTI']
all_data = all_data.dropna()

In [367]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(all_data)
all_data_scaled = pd.DataFrame(scaler.transform(all_data))

In [370]:
class Model:
    def __init__(self):
        self.data = None
        self.model = None
        
        
    
    def __build_model(self, input_shape, outputs):
        model = Sequential([
            LSTM(64, input_shape=input_shape, activation='relu'),
            Flatten(),
            Dense(outputs, activation='softmax')
        ])

        def sharpe_loss(_, y_pred):
            coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
            portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
            portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

            sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
            return K.exp(-sharpe)
        
        model.compile(loss=sharpe_loss, optimizer='adam')
        return model
    
    def get_allocations(self, data):
        
        data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
        data = data.iloc[1:]
        self.data = tf.cast(tf.constant(data), float)
        
        if self.model is None:
            self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
        fit_predict_data = data_w_ret[np.newaxis,:]        
        self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=20, shuffle=False)
        return self.model.predict(fit_predict_data)[0]

In [386]:
model = Model()
X = data[1500:1550]
coeffs = model.get_allocations(pd.DataFrame(X))

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 1.0298
Epoch 2/20
1/1 [==============================] - 0s 11ms/step - loss: 1.0250
Epoch 3/20
1/1 [==============================] - 0s 13ms/step - loss: 1.0202
Epoch 4/20
1/1 [==============================] - 0s 16ms/step - loss: 1.0155
Epoch 5/20
1/1 [==============================] - 0s 14ms/step - loss: 1.0108
Epoch 6/20
1/1 [==============================] - 0s 14ms/step - loss: 1.0061
Epoch 7/20
1/1 [==============================] - 0s 15ms/step - loss: 1.0014
Epoch 8/20
1/1 [==============================] - 0s 14ms/step - loss: 0.9966
Epoch 9/20
1/1 [==============================] - 0s 17ms/step - loss: 0.9915
Epoch 10/20
1/1 [==============================] - 0s 16ms/step - loss: 0.9862
Epoch 11/20
1/1 [==============================] - 0s 20ms/step - loss: 0.9805
Epoch 12/20
1/1 [==============================] - 0s 17ms/step - loss: 0.9746
Epoch 13/20
1/1 [==============================] - 0s 21ms/step

In [387]:
coeffs 

array([0.1348214 , 0.24569641, 0.40750936, 0.21197282], dtype=float32)